In [44]:
import ultralytics
ultralytics.__version__

'8.1.35'

In [45]:
import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.19.0


In [46]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import*
from collections import defaultdict
import supervision as sv

model=YOLO('yolov8s.pt')


area1=[(312,388),(289,390),(474,469),(497,462)]

area2=[(279,392),(250,397),(423,477),(454,469)]

In [47]:
# Finding the first frame of the video

In [48]:
import cv2

cap = cv2.VideoCapture('peoplecount1.mp4')
count = 0

while True:    
    ret, frame = cap.read()
    if not ret:
        break
    
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    if count == 1:  # Check if it's the first frame
        cv2.imwrite('first_frame_peoplecount1.jpg', frame)  # Save the first frame as 'first_frame.jpg'

    # Rest of your code...
    # (Don't forget to indent the remaining code properly)

cap.release()
cv2.destroyAllWindows()

In [49]:
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
class_list

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [50]:
cap = cv2.VideoCapture('peoplecount1.mp4')

In [51]:
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  # added this line
    px=pd.DataFrame(a).astype("float")
    print(px)

    list=[]
             
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'person' in c:
            list.append([x1,y1,x2,y2])
            print(c)


    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 1 car, 66.4ms
Speed: 2.8ms preprocess, 66.4ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)
            0          1           2           3         4    5
0  408.325378  220.69928  530.674683  339.526489  0.775767  2.0

0: 320x640 1 car, 76.4ms
Speed: 1.4ms preprocess, 76.4ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)
            0           1           2           3         4    5
0  408.272827  220.600174  530.746094  339.584747  0.779755  2.0

0: 320x640 1 car, 79.0ms
Speed: 1.4ms preprocess, 79.0ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)
           0           1           2           3        4    5
0  408.20932  220.629303  530.724976  339.577087  0.78051  2.0

0: 320x640 1 car, 60.0ms
Speed: 1.3ms preprocess, 60.0ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)
            0           1           2          3         4    5
0  408.259125  220.572327  530.743042  339.61142  0.781459  

In [52]:
area1=[(312,388),(289,390),(474,469),(497,462)]

area2=[(279,392),(250,397),(423,477),(454,469)]

In [53]:
tracker=Tracker()

count=0
cap = cv2.VideoCapture('peoplecount1.mp4')

people_entering = {}
entering = set()

people_exiting = {}
exiting = set()

   
while cap.isOpened():    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    # if count % 2 != 0:
    #     continue
    frame=cv2.resize(frame,(1020,500))
#    frame=cv2.flip(frame,1)
    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'person' in c:
            list.append([x1,y1,x2,y2])
            
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox

        ####people entering
        results = cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False)
           #print(results)
        if results >= 0:
            people_entering[id] = ((x4,y4))
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)
        if id in people_entering:
            results1 = cv2.pointPolygonTest(np.array(area1,np.int32),((x4,y4)),False)
            if results1 >= 0:
                cv2.rectangle(frame,(x3,y3),(x4,y4),(255,0,0),2)
                #cv2.circle(frame,(x4,y4),5,(255,0,255),-1)
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255),1)
                entering.add(id)
        
      

        #####people_exiting
        results2 = cv2.pointPolygonTest(np.array(area1,np.int32),((x4,y4)),False)
           #print(results2)
        if results2 >= 0:
            people_exiting[id] = ((x4,y4))
            cv2.rectangle(frame,(x3,y3),(x4,y4),(255,0,0),2)
        if id in people_exiting:
            results3 = cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False)
            if results3 >= 0:
                cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)
                #cv2.circle(frame,(x4,y4),5,(255,0,255),-1)
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,(0.5),(255,255,255),1)
                exiting.add(id)
        
            
        
    cv2.polylines(frame,[np.array(area1,np.int32)],True,(255,0,0),2)
    cv2.putText(frame,str('1'),(504,471),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)

    cv2.polylines(frame,[np.array(area2,np.int32)],True,(0,0,255),2)
    cv2.putText(frame,str('2'),(466,485),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)
    
    #print (people_entering)
    #print (len(entering))
    i = (len(entering))
    cv2.putText(frame,('entering - ') + str(i),(82,108),cv2.FONT_HERSHEY_COMPLEX,(0.7),(0,255,0),2)
    #print (people_exiting)
    #print (len(exiting)) 
    j = (len(exiting))
    cv2.putText(frame,('exiting - ') + str(j),(82, 148),cv2.FONT_HERSHEY_COMPLEX,(0.7),(255,0,255),2)
                # Write the frame with annotations to the output video
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 1 car, 70.0ms
Speed: 1.6ms preprocess, 70.0ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 73.5ms
Speed: 1.3ms preprocess, 73.5ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 72.3ms
Speed: 1.3ms preprocess, 72.3ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 73.4ms
Speed: 1.4ms preprocess, 73.4ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 68.9ms
Speed: 1.3ms preprocess, 68.9ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 71.1ms
Speed: 1.3ms preprocess, 71.1ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 car, 67.8ms
Speed: 1.8ms preprocess, 67.8ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 73.6ms
Speed: 1.2ms preprocess, 73.6ms inference, 0.6ms postprocess per image a